# Homework 1

Solve problem 2 parts b through d.

## Part 2b

Write a simulator that can randomly generate sequences of "weathers" from the state transition function. 

## Part 2c
Use your simulator to determine the stationary distribution of this Markov chain. The stationary distribution measures the probability that a random day will be sunny, cloudy, or rainy.

(i)     How to initialize the simulation

(ii)    How many transitions should be considered before taking a result

(iii)   How many simulation runs were conducted to arrive at the statistics for the stationary distribution

## Part 2d
Can you devise a closed-form solution to calculate the stationary distribution based on the state transition matrix above?

Perform the diagonalization of the state matrix A to get the same result